In [1]:
import api
import datetime
import math


In [2]:
config = api.CloudberryConfig('http://localhost', 9000)
raw_api = api.RawDataApi(config)
flux_api = api.FluxDataApi(config)

In [3]:
get_current_timestamp = lambda: math.trunc(datetime.datetime.utcnow().timestamp())

custom_bucket_name = None

measurement_name = 'notebook_demo_log'
computation_data = [
    api.ComputationData(
        time=get_current_timestamp(),
        fields={'python': 1, 'cloudberry': 2},
        tags={'evaluation_id': '3e29294d-bf7c-43c6-86b8-c9073de2685f'}
    )
]

r = raw_api.save_measurement_data(computation_data, measurement_name, custom_bucket_name)
r

True

In [ ]:
bucket_name = custom_bucket_name or "cloudberry-logs"
raw_query = f"from(bucket:\"{bucket_name}\") |> range(start:0) |> filter(fn: (r) => r.python == 1)"
result = flux_api.query(raw_query, True)
result

In [12]:
tags = {'evaluation_id': '3e29294d-bf7c-43c6-86b8-c9073de2685f'}
fields = {'python': 1}

result = raw_api.get_measurement_data(api.LogFilters(tags, fields), measurement_name, custom_bucket_name)
result

[{'result': '_result',
  'table': 0,
  '_start': 0.0,
  '_stop': 1593543314.9135404,
  '_time': 1593379990.0,
  '_measurement': 'notebook_demo_log',
  'evaluation_id': '3e29294d-bf7c-43c6-86b8-c9073de2685f',
  'cloudberry': 2,
  'python': 1},
 {'result': '_result',
  'table': 0,
  '_start': 0.0,
  '_stop': 1593543314.9135404,
  '_time': 1593380998.0,
  '_measurement': 'notebook_demo_log',
  'evaluation_id': '3e29294d-bf7c-43c6-86b8-c9073de2685f',
  'cloudberry': 2,
  'python': 1},
 {'result': '_result',
  'table': 0,
  '_start': 0.0,
  '_stop': 1593543314.9135404,
  '_time': 1593535804.0,
  '_measurement': 'notebook_demo_log',
  'evaluation_id': '3e29294d-bf7c-43c6-86b8-c9073de2685f',
  'cloudberry': 2,
  'python': 1}]

In [ ]:
df = api.as_data_frame(result)

In [13]:
import pandas as pd
pd.DataFrame(result)

,result,table,_start,_stop,_time,_measurement,evaluation_id,cloudberry,python
0,_result,0,0.0,1.593543e+09,1.593380e+09,notebook_demo_log,3e29294d-bf7c-43c6-86b8-c9073de2685f,2,1
1,_result,0,0.0,1.593543e+09,1.593381e+09,notebook_demo_log,3e29294d-bf7c-43c6-86b8-c9073de2685f,2,1
2,_result,0,0.0,1.593543e+09,1.593536e+09,notebook_demo_log,3e29294d-bf7c-43c6-86b8-c9073de2685f,2,1


In [33]:
raw_api.delete_measurement_data('workplace_log')

True

In [ ]:
# File upload test

uploader = api.FileUploader(config)
response = uploader.upload_file('emas-20190412T120536.log')
print(response)

